## Contents taken from Udacity tensorflow-2 course's Lesson 9.

# Word Embeddings and Sentiment

In this colab, you'll work with word embeddings and train a basic recurrent neural network to predict text sentiment. At the end, you'll be able to visualize how the network sees the related sentiment of each word in the dataset.

## Import TensorFlow and related functions

In [1]:
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## Get the dataset

We're going to use a dataset containing Amazon and Yelp reviews, with their related sentiment (1 for positive, 0 for negative). This dataset was originally extracted from [here](https://www.kaggle.com/marklvl/sentiment-labelled-sentences-data-set).

In [2]:
!wget --no-check-certificate \
    -O /tmp/sentiment.csv https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P

--2020-08-07 11:27:25--  https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P
Resolving drive.google.com (drive.google.com)... 172.217.218.101, 172.217.218.113, 172.217.218.102, ...
Connecting to drive.google.com (drive.google.com)|172.217.218.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/i2t5jtrhemee57u8n48k94i8ld14phmb/1596799575000/11118900490791463723/*/13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P [following]
--2020-08-07 11:27:26--  https://doc-08-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/i2t5jtrhemee57u8n48k94i8ld14phmb/1596799575000/11118900490791463723/*/13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P
Resolving doc-08-ak-docs.googleusercontent.com (doc-08-ak-docs.googleusercontent.com)... 108.177.126.132, 2a00:1450:4013:c01::84
Connecting to doc-08-ak-docs.googleusercontent.com (doc-08-ak-docs.googleusercontent.com)|1

In [3]:
import numpy as np
import pandas as pd

dataset = pd.read_csv('/tmp/sentiment.csv')

sentences = dataset['text'].tolist()
labels = dataset['sentiment'].tolist()

# Separate out the sentences and labels into training and test sets
training_size = int(len(sentences) * 0.8)

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

# Make labels into numpy arrays for use with the network later
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

print("Example training inputs and outputs")
for i in range(5):
  print(training_sentences[i], "---->" , training_labels[i])


Example training inputs and outputs
So there is no way for me to plug it in here in the US unless I go by a converter. ----> 0
Good case Excellent value. ----> 1
Great for the jawbone. ----> 1
Tied to charger for conversations lasting more than 45 minutes.MAJOR PROBLEMS!! ----> 0
The mic is great. ----> 1


## Tokenize the dataset

Tokenize the dataset, including padding and OOV, we'll follow the same process as we did in tutorial-1 on text_processing.

In [4]:
vocab_size = 1000 ## Maximum vocabulary size, will consider only 1000 most 
                  ## occuring words and assign <OOV> id to everything else

embedding_dim = 16
max_length = 100  ## Maximum number of input tokens our architecture will read
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
BATCH_SIZE=128


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, padding=padding_type, 
                       truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length, 
                               padding=padding_type, truncating=trunc_type)



## Prepare tf.dataset

In [5]:
dataset = tf.data.Dataset.from_tensor_slices((padded, training_labels))
dataset = dataset.batch(BATCH_SIZE)

example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([128, 100]), TensorShape([128]))

## Train a Basic Sentiment Model with Embeddings

The basic model looks like below figure:

<p align="center">
  <img src="https://cdn.analyticsvidhya.com/wp-content/uploads/2019/01/Screenshot-from-2019-01-18-15-51-14.png" />
</p>

*E* = represents embedding layer (converts token_id to a vector of length embedding_dim)  
*a <sup>\<i\></sup>* = Hidden state propogating in RNN network  
*y_hat* = classfication output

In [6]:
# Build a basic sentiment network
# Note the embedding layer is first, 
# and the output is only 1 node as it is either 0 or 1 (negative or positive)


## tf.keras.Sequential groups a linear stack of layers into a tf.keras.Model.



model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.SimpleRNN(embedding_dim),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Show model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 16)           16000     
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 16)                528       
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 16,545
Trainable params: 16,545
Non-trainable params: 0
_________________________________________________________________


### We can see that the model summary follows our architecture's image. 

In [7]:
num_epochs = 10
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
50/50 [==============================] - 4s 71ms/step - loss: 0.6925 - accuracy: 0.5229 - val_loss: 0.6955 - val_accuracy: 0.4987
Epoch 2/10
50/50 [==============================] - 3s 67ms/step - loss: 0.6369 - accuracy: 0.6842 - val_loss: 0.6980 - val_accuracy: 0.5664
Epoch 3/10
50/50 [==============================] - 3s 67ms/step - loss: 0.5244 - accuracy: 0.7815 - val_loss: 0.6954 - val_accuracy: 0.6015
Epoch 4/10
50/50 [==============================] - 3s 66ms/step - loss: 0.4414 - accuracy: 0.8267 - val_loss: 0.7419 - val_accuracy: 0.6040
Epoch 5/10
50/50 [==============================] - 3s 68ms/step - loss: 0.3956 - accuracy: 0.8493 - val_loss: 0.8135 - val_accuracy: 0.5865
Epoch 6/10
50/50 [==============================] - 3s 69ms/step - loss: 0.3329 - accuracy: 0.8839 - val_loss: 0.8785 - val_accuracy: 0.5815
Epoch 7/10
50/50 [==============================] - 3s 67ms/step - loss: 0.3050 - accuracy: 0.8939 - val_loss: 0.8585 - val_accuracy: 0.5940
Epoch 8/10
50

The model didn't do very well on this data and it's understandable as the architecture is fairly small. But the idea was to show how to use *tf.keras.Sequential() API* to build a failry *simple many-to-one RNN acrhitecture*

## Get files for visualizing the network

The code below will download two files for visualizing how your network "sees" the sentiment related to each word. Head to http://projector.tensorflow.org/ and load these files, then click the "Sphereize" checkbox.

In [8]:
# First get the weights of the embedding layer
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(1000, 16)


In [ ]:
import io

# Write out the embedding vectors and metadata
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [ ]:
# Download the files
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

## Predicting Sentiment in New Reviews

Now that you've trained and visualized your network, take a look below at how we can predict sentiment in new reviews the network has never seen before.

In [9]:
# Use the model to predict a review   
fake_reviews = ['I love this phone', 'I hate spaghetti', 
                'Everything was cold',
                'Everything was hot exactly as I wanted', 
                'Everything was green', 
                'the host seated us immediately',
                'they gave us free chocolate cake', 
                'not sure about the wilted flowers on the table',
                'only works when I stand on tippy toes', 
                'does not work when I stand on my head']

print(fake_reviews) 

# Create the sequences
padding_type='post'
sample_sequences = tokenizer.texts_to_sequences(fake_reviews)
fakes_padded = pad_sequences(sample_sequences, padding=padding_type, maxlen=max_length)           

print('\nHOT OFF THE PRESS! HERE ARE SOME NEWLY MINTED, ABSOLUTELY GENUINE REVIEWS!\n')              

classes = model.predict(fakes_padded)

# The closer the class is to 1, the more positive the review is deemed to be
for x in range(len(fake_reviews)):
  print(fake_reviews[x])
  print(classes[x])
  print('\n')

# Try adding reviews of your own
# Add some negative words (such as "not") to the good reviews and see what happens
# For example:
# they gave us free chocolate cake and did not charge us

['I love this phone', 'I hate spaghetti', 'Everything was cold', 'Everything was hot exactly as I wanted', 'Everything was green', 'the host seated us immediately', 'they gave us free chocolate cake', 'not sure about the wilted flowers on the table', 'only works when I stand on tippy toes', 'does not work when I stand on my head']

HOT OFF THE PRESS! HERE ARE SOME NEWLY MINTED, ABSOLUTELY GENUINE REVIEWS!

I love this phone
[0.96117866]


I hate spaghetti
[0.02996874]


Everything was cold
[0.8984161]


Everything was hot exactly as I wanted
[0.94505495]


Everything was green
[0.514217]


the host seated us immediately
[0.9449767]


they gave us free chocolate cake
[0.92621785]


not sure about the wilted flowers on the table
[0.12957855]


only works when I stand on tippy toes
[0.76650923]


does not work when I stand on my head
[0.02863884]


